# Chapter3, Vector Spaces and Subspaces

## 3.1 Spaces of Vectors

이 Chapter는 Vector Space와 Subspace의 정의에 대해 다루기에 구현할 부분이 마땅치 않아 Skip했습니다.

## 3.2 The NullSpace of A : Ax = 0 & 3.3 The rank and the Row Reduced Form

이 Chapter에서는 Nullspace의 정의와 이를 구하는 방법에 대해 나옵니다.
Nullspace를 구하는 알고리즘을 구현해보겠습니다.

전체적으로 Rref를 만드는 Elimination 과정과 비슷하며 앞에서 이용했던 Elimination1을 조금 수정했습니다.

다만, Back subtitution을 이용한 방법은 앞에서 소개한 방법과 거의 흡사해, 3.3절의 Rref에서 Nullspace로 변환하는 알고리즘을 이용해서 구현해보았습니다. 

위 코드는 pivot_cols, free_cols, Rref, Nullspace를 모두 한번에 구할 수 있으며, 그에 따라 pivot_cols의 개수인 rank까지 구할 수 있습니다.

In [23]:
import sys

def make_rref(matrix, pivot_cols):
    col_of_pivot = 0
    i = 0
    
    while i < len(matrix) and col_of_pivot < len(matrix[0]):
        pivot = matrix[i][col_of_pivot]
        
        if pivot == 0:#0인 경우 pivot이 아니니 다음 열로 넘어가 탐색한다.
            col_of_pivot += 1
        else:
            pivot_cols.append(col_of_pivot)# pivot이니 pivot_cols에 저장한다.
            for j in range(i + 1, len(matrix)):
                multiplier = matrix[j][col_of_pivot] / pivot
                for k in range(col_of_pivot, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i += 1  # 다음 행으로 이동
            col_of_pivot += 1
    #위에서 아래로 elimination
    
    
    index = 0
    for pivot_col in pivot_cols:
        if matrix[index][pivot_col] != 1:
            multiplier = matrix[index][pivot_col]
            for col in range(len(matrix[0])):
                matrix[index][col] /= multiplier
        index+=1
    #pivot 값 모두 1로 만들기
    
    pivot_cols.reverse()
    
    index = 0
    i = len(matrix) -1
    while i >= 0 and index<len(pivot_cols):
        pivot = matrix[i][pivot_cols[index]]
        
        if pivot != 1:#1인 경우 pivot이 아니니 다음 행로 넘어가 탐색한다.
            i -= 1
        else:
            for j in range(i - 1, -1, -1):
                multiplier = matrix[j][pivot_cols[index]] / pivot
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i -= 1  # 다음 열으로 이동
            index += 1
    
    #아래에서 위로 elimination

    
    return matrix
    #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다
    
def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
    
def make_nullspace(matrix, pivot_cols, free_cols):
    cols = len(matrix[0]) - len(pivot_cols)
    rows = len(matrix[0])
    #Nullspace는 원 matrix의 rows, cols수가 반대
    
    nullspace = list()
    
    for _ in range(rows):
        temp = [0]*cols
        nullspace.append(temp)
    
    free_index = 0
    pivot_index = 0
    index = 0
    
    for row in range(rows):
        if row in pivot_cols:
            index = 0
            for col in free_cols:
                nullspace[row][index]= -matrix[pivot_index][col]
                index+=1
            pivot_index += 1
            #Nullspace의 행이 not free하다면 RRef의 같은 행의 free columns들을 가져옴
        else:
            nullspace[row][free_index] = 1
            free_index+=1
            #아니라면 Identity matrix처럼 1로 만듦
            
    return nullspace

"""
    위 알고리즘은 R = | I F | 일 때, N = | -F | 를 가진다는 관계를 통해 구현되었습니다.
                      | 0 0 |            |  I |
"""

    
matrix = list()

rows = int(input("Matrix A 행 수를 입력하세요: "))
cols = int(input("Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
print()


pivot_cols = list() #pivot column들의 열번호를 저장한다.
make_rref(matrix, pivot_cols)
free_cols = [x for x in range(0, cols) if x not in pivot_cols]# free columns들의 열번호를 저장한다.

print("rref of matrix")
print_matrix(matrix)
print()

print("pivot_cols의 열 번호 : ")
print(pivot_cols)
print()

print("free_cols의 열 번호 :")
print(free_cols)
print()

N_matrix = make_nullspace(matrix, pivot_cols, free_cols)

print("Nullspace : ")
print_matrix(N_matrix)
print()





Matrix A 행 수를 입력하세요: 3
Matrix A 열 수를 입력하세요: 5
행 1을 공백으로 구분하여 입력하세요: 1 3 0 2 -1
행 2을 공백으로 구분하여 입력하세요: 0 0 1 4 -3
행 3을 공백으로 구분하여 입력하세요: 1 3 1 6 -4

rref of matrix
1.0	3.0	0.0	2.0	-1.0	
0.0	0.0	1.0	4.0	-3.0	
0.0	0.0	0.0	0.0	0.0	

pivot_cols의 열 번호 : 
[2, 0]

free_cols의 열 번호 :
[1, 3, 4]

Nullspace : 
-3.0	-2.0	1.0	
1	0	0	
-0.0	-4.0	3.0	
0	1	0	
0	0	1	



## 3.4 The Complete Solution to Ax = b


이 Chapter에서는 r, m, n의 관계를 통해 해가 어떻게 존재할 지 다룹니다. 

r = m and r= n : 1 solution

r = m and r < n : infinite solutions

r < m and r = n : 0 or 1 solution

r < m and r < n : 0 or infinite solutions

Nullspace에 대해서 구하는 것은 구현하였기에, Particular Solution을 구해보겠습니다.

In [62]:
import sys

def make_rref(matrix, pivot_cols):
    col_of_pivot = 0
    i = 0
    
    while i < len(matrix) and col_of_pivot < len(matrix[0])-1:
        pivot = matrix[i][col_of_pivot]
        
        if pivot == 0:#0인 경우 pivot이 아니니 다음 열로 넘어가 탐색한다.
            col_of_pivot += 1
        else:
            pivot_cols.append(col_of_pivot)# pivot이니 pivot_cols에 저장한다.
            for j in range(i + 1, len(matrix)):
                multiplier = matrix[j][col_of_pivot] / pivot
                for k in range(col_of_pivot, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i += 1  # 다음 행으로 이동
            col_of_pivot += 1
    #위에서 아래로 elimination
    
    
    index = 0
    for pivot_col in pivot_cols:
        if matrix[index][pivot_col] != 1:
            multiplier = matrix[index][pivot_col]
            for col in range(len(matrix[0])):
                matrix[index][col] /= multiplier
        index+=1
    #pivot 값 모두 1로 만들기
    
    pivot_cols.reverse()
    
    index = 0
    i = len(matrix) -1
    while i >= 0 and index<len(pivot_cols):
        pivot = matrix[i][pivot_cols[index]]
        
        if pivot != 1:#1인 경우 pivot이 아니니 다음 행로 넘어가 탐색한다.
            i -= 1
        else:
            for j in range(i - 1, -1, -1):
                multiplier = matrix[j][pivot_cols[index]] / pivot
                for k in range(0, len(matrix[0])):
                    matrix[j][k] -= (matrix[i][k] * multiplier)
            i -= 1  # 다음 열으로 이동
            index += 1
    
    #아래에서 위로 elimination

    pivot_cols.reverse()
    
    return matrix
    #성공적으로 Elimination을 했다면 True를 반환해 Elimination된 Matrix를 출력한다
    
def print_matrix(matrix):
    rows = len(matrix)
    cols = len(matrix[0])
    
    for i in range(rows):
        for j in range(cols):
            print(matrix[i][j], end ="\t")
        print()
    
def make_nullspace(matrix, pivot_cols, free_cols):
    cols = len(matrix[0]) - len(pivot_cols) - 1
    rows = len(matrix[0]) - 1
    #Nullspace는 원 matrix의 rows, cols수가 반대
    
    nullspace = list()
    
    for _ in range(rows):
        temp = [0]*cols
        nullspace.append(temp)
    
    free_index = 0
    pivot_index = 0
    index = 0
    
    for row in range(rows):
        if row in pivot_cols:
            index = 0
            for col in free_cols:
                nullspace[row][index]= -matrix[pivot_index][col]
                index+=1
            pivot_index += 1
            #Nullspace의 행이 not free하다면 RRef의 같은 행의 free columns들을 가져옴
        else:
            nullspace[row][free_index] = 1
            free_index+=1
            #아니라면 Identity matrix처럼 1로 만듦
            
    return nullspace

"""
    위 알고리즘은 R = | I F | 일 때, N = | -F | 를 가진다는 관계를 통해 구현되었습니다.
                      | 0 0 |            |  I |
"""

def find_particular(matrix, pivot_cols):
    answer = [0]*(len(matrix[0])-1)
    
    target = list()
    
    for i in range(len(matrix)):
        target.append(matrix[i][len(matrix[0])-1])
    #augmented matrix의 마지막 부분(Rx = b에서 b만 저장)
    
    index = 0
    i = 0
    
    while True:
            
        flag = True
        for j in range(len(matrix[0])-1):
            if matrix[i][j] != 0:
                flag = False
                break
            #만약 rref의 특정 행의 모든 열이 0이면 True, 아니면 False
            
        if flag and target[i] != 0: #rref의 특정 행의 모든 열이 0인데, b부분의 값이 0이 아닌경우
            return None #해가 없으므로 None 반환
        
        else:
            if i == len(matrix) -1 and index == len(matrix[0])-2 :
                break
            
            if index in pivot_cols:
                answer[index] = target[i] 
                if index +1 < len(matrix[0]) -1 :
                    index+=1
                if i +1 < len(matrix) :
                    i+=1
            else:
                answer[index] = 0
                if index +1 < len(matrix[0]) -1 :
                    index+=1

    return answer

matrix = list()

rows = int(input("Augmented Matrix A 행 수를 입력하세요: "))
cols = int(input("Augmented Matrix A 열 수를 입력하세요: "))

for i in range(rows):
        row = input(f"행 {i+1}을 공백으로 구분하여 입력하세요: ").split()
        if len(row) != cols:
            print("열 수와 입력된 값의 개수가 일치하지 않습니다.")
            sys.exit(1)
        else:
            matrix.append([float(val) for val in row])
print()


pivot_cols = list() #pivot column들의 열번호를 저장한다.
make_rref(matrix, pivot_cols)
free_cols = [x for x in range(0, cols-1) if x not in pivot_cols]# free columns들의 열번호를 저장한다.

print("rref of matrix")
print_matrix(matrix)
print()

print("pivot_cols의 열 번호 : ")
print(pivot_cols)
print()

print("free_cols의 열 번호 :")
print(free_cols)
print()

N_matrix = make_nullspace(matrix, pivot_cols, free_cols)

print("Nullspace : ")
print_matrix(N_matrix)
print()


r = len(pivot_cols)
m = rows
n = cols -1
particular_solution = find_particular(matrix, pivot_cols)

if r == m and r == n:
    print("particular solution : ")
    print(particular_solution)
    print()
    print("There is only one Nullspace")

elif r == m and r < n:
    print("particular solution : ")
    print(particular_solution)
    print()
    print("There are infinite combination of Nullspace")

elif r < m and r== n:
    if particular_solution :
        print("particular solution : ")
        print(particular_solution)
        print()
        print("There is only one Nullspace")
    else:
        print("There is no solution")
    
elif r < m and r < n :
    if particular_solution :
        print("particular solution : ")
        print(particular_solution)
        print()
        print("There are infinite combination of Nullspace")
    else:
        print("There is no solution")

Augmented Matrix A 행 수를 입력하세요: 3
Augmented Matrix A 열 수를 입력하세요: 3
행 1을 공백으로 구분하여 입력하세요: 1 0 6
행 2을 공백으로 구분하여 입력하세요: 1 1 0
행 3을 공백으로 구분하여 입력하세요: 1 2 -6

rref of matrix
1.0	0.0	6.0	
0.0	1.0	-6.0	
0.0	0.0	0.0	

pivot_cols의 열 번호 : 
[0, 1]

free_cols의 열 번호 :
[]

Nullspace : 



particular solution : 
[6.0, -6.0]

There is only one Nullspace


## 3.5 Independence, Basis and Dimension & 3.6 Dimensions of the Four Subspaces

이 Chapter에서는 Independence, Basis, Dimension에 대해 다룹니다.

Linear Independence란 Ax = 0 에서 x가 오직 0 벡터인 경우를 말합니다. 이는 위의 코드로 Nullspace를 구해서 len()를 구현만 하면 간단히 0벡터인지 알 수 있습니다.

다음은 Basis인데, Basis vectors란 Linear independence하며 space를 모두 span할 수 있는 경우를 말합니다. 복잡해 보이지만, 대수적으로 Elimination 후 pivot column 혹은 row만 찾아내면 되기에 역시 위의 코드에서 모두 구해낼 수 있습니다.

마지막은 Dimension, Basis의 개수인데 위에서 구해낸 Basis를 확인하면 쉽게 구할 수 있을 뿐더러, 3.6절에 의하면

dim(Row space) = r
dim(Column Space) = r
dim(Nullspace) = n-r
dim(leftNullspace) = m-r

이라는 공식이 있기에 특별한 추가 구현없이 모두 확인이 가능하므로 Skip하겠습니다.